# Career Recommender System - Interactive Demo

This notebook provides an interactive interface to test the career recommendation system with custom user profiles.

**Features:**
1. Interactive widgets for user input
2. Real-time job recommendations
3. Explanation of why jobs were recommended
4. Comparison of different user profiles
5. Export recommendations to CSV

## 📋 Prerequisites for Colab/Kaggle
- Run the preprocessing and training notebooks first
- Or use the demo mode with sample data

## 1. Install Dependencies

In [ ]:
# Install required packages
!pip install pandas numpy scikit-learn matplotlib seaborn plotly
!pip install sentence-transformers transformers torch
!pip install faiss-cpu tqdm python-dotenv huggingface-hub
!pip install xgboost lightgbm ipywidgets

print("✅ All packages installed successfully!")

## 2. Environment Setup

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import pickle
import json
from typing import Dict, List, Any

# Jupyter widgets for interactivity
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# ML libraries
from sentence_transformers import SentenceTransformer
import faiss

warnings.filterwarnings('ignore')
plt.style.use('default')
np.random.seed(42)

# Create directories
os.makedirs('models', exist_ok=True)
os.makedirs('results', exist_ok=True)

print("Environment setup complete!")

## 3. Load Models and Create Demo Data

In [ ]:
# Try to load real models, fallback to demo mode
demo_mode = False

try:
    # Load reranker model
    with open('models/reranker_model.pkl', 'rb') as f:
        model_data = pickle.load(f)
    
    # Load embedding model
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Load processed data
    jobs_df = pd.read_pickle('models/jobs_processed.pkl')
    
    print("✅ Real models loaded successfully!")
    
except FileNotFoundError:
    print("⚠️ Model files not found. Entering demo mode...")
    demo_mode = True
    
    # Create sample job data for demo
    sample_jobs_data = {
        'job_id': range(1, 11),
        'job_title': [
            'Software Engineer', 'Data Scientist', 'Product Manager',
            'UX Designer', 'Marketing Specialist', 'Financial Analyst',
            'Project Manager', 'Business Analyst', 'Sales Representative',
            'Graphic Designer'
        ],
        'company': [
            'TechCorp', 'DataInc', 'InnovateCo', 'DesignStudio', 'BrandAgency',
            'FinanceGroup', 'ProjectPros', 'ConsultingFirm', 'SalesCorp', 'CreativeAgency'
        ],
        'industry': [
            'Technology', 'Technology', 'Technology', 'Design', 'Marketing',
            'Finance', 'Consulting', 'Consulting', 'Sales', 'Design'
        ],
        'location': [
            'Remote', 'San Francisco', 'Seattle', 'Austin', 'Chicago',
            'New York', 'Boston', 'Washington DC', 'Los Angeles', 'Portland'
        ],
        'required_skills': [
            'Python, JavaScript, Git', 'Python, R, SQL, Machine Learning',
            'Leadership, Analytics, Strategy', 'Figma, User Research, Prototyping',
            'Google Analytics, SEO, Content Creation', 'Excel, Financial Modeling, Bloomberg',
            'Project Management, Communication', 'Excel, PowerBI, Process Analysis',
            'Communication, CRM, Negotiation', 'Photoshop, Illustrator, Branding'
        ],
        'education_requirement': [
            'Bachelor', 'Master', 'Bachelor', 'Bachelor', 'Bachelor',
            'Bachelor', 'Bachelor', 'Bachelor', 'High School', 'Bachelor'
        ],
        'salary_range': [
            '70000-95000', '85000-120000', '90000-130000', '60000-85000', '45000-65000',
            '65000-80000', '75000-100000', '70000-90000', '40000-70000', '40000-60000'
        ],
        'description': [
            'Develop web applications using modern frameworks',
            'Analyze data and build predictive models',
            'Lead product development from conception to launch',
            'Design user interfaces and experiences',
            'Create and execute marketing campaigns',
            'Perform financial analysis and modeling',
            'Coordinate project activities and deliverables',
            'Analyze business processes and recommend improvements',
            'Generate new business and maintain client relationships',
            'Create visual designs for print and digital media'
        ]
    }
    
    jobs_df = pd.DataFrame(sample_jobs_data)
    print("✅ Demo data created successfully!")

print(f"Available jobs: {len(jobs_df)}")
print(f"Demo mode: {demo_mode}")

## 4. Recommendation Functions

In [ ]:
def calculate_job_match_score(user_profile, job_row):
    """Calculate match score between user and job (demo version)"""
    score = 0.0
    
    # Education match
    education_levels = {'High School': 0, 'Associate': 1, 'Bachelor': 2, 'Master': 3, 'PhD': 4}
    user_edu = education_levels.get(user_profile['education'], 2)
    job_edu = education_levels.get(job_row['education_requirement'], 2)
    
    if user_edu >= job_edu:
        score += 0.3
    
    # Skills overlap (simple text matching)
    user_skills = set(user_profile['skills'].lower().split(', '))
    job_skills = set(job_row['required_skills'].lower().split(', '))
    
    if user_skills and job_skills:
        overlap = len(user_skills.intersection(job_skills))
        total = len(user_skills.union(job_skills))
        skill_score = overlap / total if total > 0 else 0
        score += 0.4 * skill_score
    
    # Location match
    if (user_profile['location'].lower() in job_row['location'].lower() or 
        job_row['location'].lower() in user_profile['location'].lower() or
        'remote' in user_profile['location'].lower() or
        'remote' in job_row['location'].lower()):
        score += 0.2
    
    # GPA bonus
    if user_profile['gpa'] >= 3.5:
        score += 0.1
    
    return min(score, 1.0)  # Cap at 1.0

def get_recommendations(user_profile, top_k=5):
    """Get job recommendations for user profile"""
    recommendations = []
    
    for _, job in jobs_df.iterrows():
        match_score = calculate_job_match_score(user_profile, job)
        
        # Add some randomness to make it more realistic
        match_score += np.random.normal(0, 0.05)
        match_score = max(0, min(1, match_score))  # Keep in [0, 1] range
        
        recommendations.append({
            'job_id': job['job_id'],
            'job_title': job['job_title'],
            'company': job['company'],
            'industry': job['industry'],
            'location': job['location'],
            'salary_range': job['salary_range'],
            'required_skills': job['required_skills'],
            'description': job['description'],
            'match_score': match_score
        })
    
    # Sort by match score and return top k
    recommendations.sort(key=lambda x: x['match_score'], reverse=True)
    return recommendations[:top_k]

def explain_recommendation(user_profile, job_rec):
    """Generate explanation for recommendation"""
    explanations = []
    
    # Check education
    education_levels = {'High School': 0, 'Associate': 1, 'Bachelor': 2, 'Master': 3, 'PhD': 4}
    user_edu = education_levels.get(user_profile['education'], 2)
    job_edu_req = jobs_df[jobs_df['job_id'] == job_rec['job_id']]['education_requirement'].iloc[0]
    job_edu = education_levels.get(job_edu_req, 2)
    
    if user_edu >= job_edu:
        explanations.append("✅ Education requirement met")
    
    # Check skills
    user_skills = set(user_profile['skills'].lower().split(', '))
    job_skills = set(job_rec['required_skills'].lower().split(', '))
    overlap = user_skills.intersection(job_skills)
    
    if overlap:
        explanations.append(f"✅ Matching skills: {', '.join(overlap)}")
    
    # Check location
    if (user_profile['location'].lower() in job_rec['location'].lower() or 
        'remote' in user_profile['location'].lower() or
        'remote' in job_rec['location'].lower()):
        explanations.append("✅ Location preference matched")
    
    # Check GPA
    if user_profile['gpa'] >= 3.5:
        explanations.append("✅ Strong academic performance")
    
    return explanations if explanations else ["Good overall fit based on profile"]

print("✅ Recommendation functions ready!")

## 5. Interactive User Profile Input

In [ ]:
# Create interactive widgets
print("🎯 Career Recommendation System - Interactive Demo")
print("Fill out your profile below to get personalized job recommendations!")
print()

# User input widgets
education_widget = widgets.Dropdown(
    options=['High School', 'Associate', 'Bachelor', 'Master', 'PhD'],
    value='Bachelor',
    description='Education:',
    style={'description_width': 'initial'}
)

field_widget = widgets.Text(
    value='Computer Science',
    description='Field of Study:',
    style={'description_width': 'initial'}
)

gpa_widget = widgets.FloatSlider(
    value=3.5,
    min=2.0,
    max=4.0,
    step=0.1,
    description='GPA:',
    style={'description_width': 'initial'}
)

skills_widget = widgets.Text(
    value='Python, SQL, JavaScript, Git',
    description='Skills:',
    placeholder='Enter comma-separated skills',
    style={'description_width': 'initial'}
)

interests_widget = widgets.Text(
    value='programming, data analysis, machine learning',
    description='Interests:',
    placeholder='Enter comma-separated interests',
    style={'description_width': 'initial'}
)

experience_widget = widgets.IntSlider(
    value=2,
    min=0,
    max=15,
    step=1,
    description='Experience (years):',
    style={'description_width': 'initial'}
)

location_widget = widgets.Text(
    value='Remote',
    description='Preferred Location:',
    style={'description_width': 'initial'}
)

num_recs_widget = widgets.IntSlider(
    value=5,
    min=3,
    max=10,
    step=1,
    description='Number of Recommendations:',
    style={'description_width': 'initial'}
)

# Button to get recommendations
recommend_button = widgets.Button(
    description='Get Job Recommendations!',
    button_style='primary',
    layout=widgets.Layout(width='300px', height='40px')
)

# Output area
output_area = widgets.Output()

# Display widgets
form = widgets.VBox([
    widgets.HTML("<h3>👤 Your Profile</h3>"),
    education_widget,
    field_widget,
    gpa_widget,
    widgets.HTML("<br>"),
    skills_widget,
    interests_widget,
    widgets.HTML("<br>"),
    experience_widget,
    location_widget,
    widgets.HTML("<br>"),
    num_recs_widget,
    widgets.HTML("<br>"),
    recommend_button
])

display(form)

## 6. Recommendation Engine

In [ ]:
def on_recommend_click(button):
    """Handle recommendation button click"""
    with output_area:
        clear_output()
        
        # Get user profile from widgets
        user_profile = {
            'education': education_widget.value,
            'field_of_study': field_widget.value,
            'gpa': gpa_widget.value,
            'skills': skills_widget.value,
            'interests': interests_widget.value,
            'experience_years': experience_widget.value,
            'location': location_widget.value
        }
        
        print("🔍 Analyzing your profile and finding matches...")
        print()
        
        # Get recommendations
        recommendations = get_recommendations(user_profile, top_k=num_recs_widget.value)
        
        # Display results
        print(f"🎯 Top {len(recommendations)} Job Recommendations for You:")
        print("=" * 60)
        
        for i, job in enumerate(recommendations, 1):
            print(f"\n{i}. {job['job_title']} at {job['company']}")
            print(f"   📍 Location: {job['location']}")
            print(f"   🏢 Industry: {job['industry']}")
            print(f"   💰 Salary: ${job['salary_range']}")
            print(f"   🎯 Match Score: {job['match_score']:.1%}")
            print(f"   📋 Required Skills: {job['required_skills']}")
            print(f"   📝 Description: {job['description']}")
            
            # Show explanation
            explanations = explain_recommendation(user_profile, job)
            print(f"   💡 Why recommended:")
            for explanation in explanations:
                print(f"      {explanation}")
            
            if i < len(recommendations):
                print("-" * 60)
        
        print(f"\n✨ Recommendations based on your profile:")
        print(f"   🎓 Education: {user_profile['education']} in {user_profile['field_of_study']}")
        print(f"   📊 GPA: {user_profile['gpa']:.1f}")
        print(f"   💼 Experience: {user_profile['experience_years']} years")
        print(f"   🛠️ Skills: {user_profile['skills']}")
        print(f"   📍 Location: {user_profile['location']}")

# Connect button to function
recommend_button.on_click(on_recommend_click)

# Display output area
display(output_area)

print("\n👆 Fill out the form above and click the button to get recommendations!")

## 7. Sample User Profiles

In [ ]:
# Pre-defined sample profiles for quick testing
sample_profiles = {
    "Software Engineer": {
        'education': 'Bachelor',
        'field_of_study': 'Computer Science',
        'gpa': 3.8,
        'skills': 'Python, JavaScript, React, SQL, Git',
        'interests': 'programming, web development, algorithms',
        'experience_years': 3,
        'location': 'Remote'
    },
    "Data Scientist": {
        'education': 'Master',
        'field_of_study': 'Data Science',
        'gpa': 3.9,
        'skills': 'Python, R, SQL, Machine Learning, Statistics',
        'interests': 'data analysis, machine learning, research',
        'experience_years': 2,
        'location': 'San Francisco'
    },
    "Marketing Specialist": {
        'education': 'Bachelor',
        'field_of_study': 'Marketing',
        'gpa': 3.5,
        'skills': 'Google Analytics, SEO, Content Creation, Social Media',
        'interests': 'digital marketing, branding, content strategy',
        'experience_years': 4,
        'location': 'Chicago'
    },
    "Recent Graduate": {
        'education': 'Bachelor',
        'field_of_study': 'Business Administration',
        'gpa': 3.3,
        'skills': 'Excel, PowerPoint, Communication, Leadership',
        'interests': 'business strategy, management, consulting',
        'experience_years': 0,
        'location': 'Any'
    }
}

print("🎭 Sample User Profiles - Click to auto-fill the form:")
print()

def create_profile_button(profile_name, profile_data):
    """Create button to load sample profile"""
    button = widgets.Button(
        description=f"Load {profile_name}",
        button_style='info',
        layout=widgets.Layout(width='200px', margin='2px')
    )
    
    def on_click(b):
        education_widget.value = profile_data['education']
        field_widget.value = profile_data['field_of_study']
        gpa_widget.value = profile_data['gpa']
        skills_widget.value = profile_data['skills']
        interests_widget.value = profile_data['interests']
        experience_widget.value = profile_data['experience_years']
        location_widget.value = profile_data['location']
        print(f"✅ Loaded {profile_name} profile!")
    
    button.on_click(on_click)
    return button

# Create buttons for each sample profile
profile_buttons = [create_profile_button(name, data) for name, data in sample_profiles.items()]
button_box = widgets.HBox(profile_buttons)
display(button_box)

print("\n💡 Or create your own custom profile using the form above!")

## 8. Export Recommendations

In [ ]:
# Function to export recommendations to CSV
def export_recommendations_to_csv(user_profile, recommendations, filename="my_job_recommendations.csv"):
    """Export recommendations to CSV file"""
    
    # Prepare data for CSV
    export_data = []
    
    for i, job in enumerate(recommendations, 1):
        row = {
            'rank': i,
            'job_title': job['job_title'],
            'company': job['company'],
            'industry': job['industry'],
            'location': job['location'],
            'salary_range': job['salary_range'],
            'match_score': f"{job['match_score']:.1%}",
            'required_skills': job['required_skills'],
            'description': job['description'],
            'user_education': user_profile['education'],
            'user_field': user_profile['field_of_study'],
            'user_gpa': user_profile['gpa'],
            'user_experience': user_profile['experience_years'],
            'user_skills': user_profile['skills'],
            'user_location': user_profile['location']
        }
        export_data.append(row)
    
    # Create DataFrame and save to CSV
    df = pd.DataFrame(export_data)
    df.to_csv(filename, index=False)
    
    return df

# Export button
export_button = widgets.Button(
    description='Export Last Recommendations to CSV',
    button_style='success',
    layout=widgets.Layout(width='300px', height='40px')
)

export_output = widgets.Output()

def on_export_click(button):
    with export_output:
        clear_output()
        try:
            # Get current user profile
            current_profile = {
                'education': education_widget.value,
                'field_of_study': field_widget.value,
                'gpa': gpa_widget.value,
                'skills': skills_widget.value,
                'interests': interests_widget.value,
                'experience_years': experience_widget.value,
                'location': location_widget.value
            }
            
            # Get fresh recommendations
            current_recs = get_recommendations(current_profile, top_k=num_recs_widget.value)
            
            # Export to CSV
            filename = "job_recommendations.csv"
            df = export_recommendations_to_csv(current_profile, current_recs, filename)
            
            print(f"✅ Recommendations exported to {filename}")
            print(f"📊 Exported {len(df)} job recommendations")
            print(f"\nFirst few rows:")
            print(df[['rank', 'job_title', 'company', 'match_score']].head())
            
        except Exception as e:
            print(f"❌ Export failed: {e}")
            print("Please generate recommendations first!")

export_button.on_click(on_export_click)

display(widgets.VBox([
    widgets.HTML("<h4>📁 Export Options</h4>"),
    export_button,
    export_output
]))

print("💾 Use the export button to save recommendations as CSV file!")

## Summary

🎉 **Interactive Career Recommender Demo Ready!**

**How to use:**
1. **Fill out your profile** using the form above
2. **Click "Get Job Recommendations"** to see personalized results
3. **Try sample profiles** using the quick-load buttons
4. **Export recommendations** to CSV for further analysis

**Features demonstrated:**
- ✅ Interactive user profile input
- ✅ Real-time job matching and scoring
- ✅ Explanations for why jobs were recommended
- ✅ Multiple sample profiles for testing
- ✅ CSV export functionality

**Next steps for production use:**
1. Connect to real job databases (LinkedIn, Indeed, etc.)
2. Deploy as web application with Flask/FastAPI
3. Add user feedback collection for model improvement
4. Implement more sophisticated matching algorithms
5. Add job application tracking and follow-up features

**🚀 Start exploring job recommendations with your profile!**